In [7]:
import requests
import json
import pandas as pd
import re

ollama_url_gen = "http://localhost:11434/api/generate"
ollama_model_name = "phi3"

def generate_toc_prompt(subject):
    return f"""
You are an expert assistant specializing in creating structured and logical book outlines.
Your task is to generate a detailed Table of Contents for a book on the given subject.
Each chapter must have a title, and chapters can include subsections and examples where applicable.
The Table of Contents should be well-structured, logically ordered, and strictly formatted as a valid JSON object without any extra text.
Be extra cautious to place comma between items. Multiple items within one key should be like array element in valid json format.

Format:
{{
    "Table_of_Contents": [
        {{
            "Chapter 1": "Title of Chapter",
            "Sections": [
                "Section 1.1: Title of Section",
                "Section 1.2: Title of Section"
            ]
        }},
        {{
            "Chapter 2": "Title of Chapter",
            "Sections": [
                "Section 2.1: Title of Section",
                "Section 2.2: Title of Section"
            ]
        }}
    ]
}}

Subject: {subject}
"""
    payload = {"model": model, "prompt": prompt, "stream": False}
    headers = {"Content-Type": "application/json"}
    response = requests.post(ollama_url_gen, headers=headers, data=json.dumps(payload))
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    content = response.json().get("response", "")
    try:
        # Extract potential JSON array or object
        json_match = re.search(r"\{.*\}|\[.*\]", content, re.DOTALL)
        if json_match:
            raw_json = json_match.group()
            
            # Attempt to fix common issues, such as missing commas between JSON objects
            sanitized_json = re.sub(r'(\}|"[^"]*")\s*("[^"]*":)', r'\1,\2', raw_json)
            return json.loads(sanitized_json)
        else:
            raise Exception(f"Valid JSON not found in response: {content}")
    except json.JSONDecodeError as e:
        raise Exception(f"Failed to parse JSON response after sanitization: {sanitized_json}") from e

try:
    toc = generate_toc("Prompt Engineering")
    print(json.dumps(toc, indent=4))
except Exception as e:
    print(str(e))


Failed to parse JSON response: {
  "Table_of_Contents": [
    {
      "Chapter_1": {
        "Title": "Introduction to Prompt Engineering",
        "Subsections": ["What is prompt engineering?", "Importance of Effective Communication"]
      }
    },
    {
      "Chapter_2": {
        "Title": "Principles and Best Practices in Prompt Design",
        "Subsections": [
          "Understanding the target AI model's limitations", 
          "Using clear, concise language"
        ]
      }
    },
    {
      "Chapter_3": {
        "Title": "Prompt Structuring Techniques for Effective Communication",
        "Subsections": [
          "How to structure a prompt effectively?", 
          "Using examples and providing context"
        ]
      }
    },
    {
      "Chapter_4": {
        "Title": "Techniques for Dynamic Prompt Adjustment during Interaction",
        "Subsections": [
          "Adapting prompts based on AI response patterns", 
          "Identifying the need to modify a prompt 

In [ ]:
generate_toc("Prompt Engineering")

In [ ]:
import requests
import json

ollama_url_gen = "http://localhost:11434/api/generate"
ollama_model_name = "phi3"

def generate_section(chapter_title, model=ollama_model_name):
    prompt = f"""
You are a knowledgeable assistant tasked with writing detailed content for a book.
Please write content for the following chapter or section title:
Title: {chapter_title}

The content should be comprehensive, logically structured, and suitable for a technical book. Ensure it includes examples where relevant.
"""
    payload = {"model": model, "prompt": prompt, "stream": False}
    headers = {"Content-Type": "application/json"}
    response = requests.post(ollama_url_gen, headers=headers, data=json.dumps(payload))
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    return response.json().get("response", "")

# Example usage
for item in toc["Table_of_Contents"]:
    for chapter, title in item.items():
        print(f"Generating content for {chapter}: {title}")
        content = generate_section(title)
        with open(f"{chapter}.txt", "w") as file:
            file.write(content)


In [ ]:
import requests
import json
import os
from fpdf import FPDF

# Ollama API configuration
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "phi3"

# Directory to save generated content
OUTPUT_DIR = "book_content"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def generate_toc(subject):
    """Generate a Table of Contents for the given subject."""
    prompt = f"""
    You are an expert assistant skilled at creating book outlines.
    Create a detailed Table of Contents for a book on the following subject:
    Subject: {subject}
    
    Structure the response as a valid JSON object with chapters and sections:
    {{
        "Table_of_Contents": [
            {{
                "Chapter 1": "Title of Chapter",
                "Sections": [
                    "Section 1.1: Title of Section",
                    "Section 1.2: Title of Section"
                ]
            }},
            {{
                "Chapter 2": "Title of Chapter",
                "Sections": [
                    "Section 2.1: Title of Section",
                    "Section 2.2: Title of Section"
                ]
            }}
        ]
    }}
    """
    payload = {"model": MODEL_NAME, "prompt": prompt, "stream": False}
    response = requests.post(OLLAMA_URL, json=payload)
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    content = response.json().get("response", "")
    try:
        toc = json.loads(content)
        return toc
    except json.JSONDecodeError:
        raise Exception(f"Failed to parse ToC JSON: {content}")

def generate_section(title):
    """Generate content for a given chapter or section title."""
    prompt = f"""
    You are a knowledgeable assistant tasked with writing detailed content for a book.
    Please write content for the following chapter or section:
    Title: {title}
    
    The content should be comprehensive, well-structured, and professional.
    Include examples, subtopics, and technical insights where applicable.
    """
    payload = {"model": MODEL_NAME, "prompt": prompt, "stream": False}
    response = requests.post(OLLAMA_URL, json=payload)
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    return response.json().get("response", "")

def save_content_to_file(filename, content):
    """Save generated content to a file."""
    with open(os.path.join(OUTPUT_DIR, filename), "w") as f:
        f.write(content)

def create_pdf_from_content(toc, output_file="Generated_Book.pdf"):
    """Generate a PDF book from the content."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_left_margin(15)
    pdf.set_right_margin(15)
    
    # Title Page
    pdf.add_page()
    pdf.set_font("Arial", size=24, style="B")
    pdf.cell(0, 10, "Generated Book", ln=True, align="C")
    pdf.ln(20)
    
    # Table of Contents
    pdf.set_font("Arial", size=16, style="B")
    pdf.cell(0, 10, "Table of Contents", ln=True)
    pdf.ln(10)
    pdf.set_font("Arial", size=12)
    for chapter in toc["Table_of_Contents"]:
        for chapter_title, sections in chapter.items():
            pdf.cell(0, 10, chapter_title, ln=True)
            if isinstance(sections, dict):
                for section_title in sections.get("Sections", []):
                    pdf.cell(10, 10, f"  {section_title}", ln=True)
            pdf.ln(5)
    
    # Chapters
    for chapter in toc["Table_of_Contents"]:
        for chapter_title, sections in chapter.items():
            pdf.add_page()
            pdf.set_font("Arial", size=18, style="B")
            pdf.cell(0, 10, chapter_title, ln=True)
            pdf.ln(10)
            
            filename = f"{chapter_title.replace(' ', '_')}.txt"
            file_path = os.path.join(OUTPUT_DIR, filename)
            if os.path.exists(file_path):
                with open(file_path, "r") as f:
                    content = f.read()
                pdf.set_font("Arial", size=12)
                pdf.multi_cell(0, 10, content)
    
    pdf.output(output_file)

# Main Execution
if __name__ == "__main__":
    subject = "Prompt Engineering"
    print(f"Generating Table of Contents for subject: {subject}")
    
    # Step 1: Generate ToC
    toc = generate_toc(subject)
    print("Table of Contents generated successfully.")
    
    # Step 2: Generate content for each chapter/section
    for chapter in toc["Table_of_Contents"]:
        for chapter_title, sections in chapter.items():
            print(f"Generating content for {chapter_title}...")
            chapter_content = generate_section(chapter_title)
            save_content_to_file(f"{chapter_title.replace(' ', '_')}.txt", chapter_content)
            
            if "Sections" in sections:
                for section_title in sections["Sections"]:
                    print(f"Generating content for {section_title}...")
                    section_content = generate_section(section_title)
                    save_content_to_file(f"{section_title.replace(' ', '_')}.txt", section_content)
    
    # Step 3: Create PDF
    print("Merging content into a PDF book...")
    create_pdf_from_content(toc)
    print("PDF book generated successfully: Generated_Book.pdf")
